[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/takazawa/PyOptBookMeijiUnivBA/blob/main/misc/zemi_sample.ipynb)

In [ ]:
# Google Colabで実行する場合は初回に必ず実行する
import os
if 'COLAB_GPU' in os.environ:
    if not os.path.exists("PyOptBookMeijiUnivBA"):
        !git clone https://github.com/takazawa/PyOptBookMeijiUnivBA.git
        !cp PyOptBookMeijiUnivBA/misc/*.csv .
    !pip install -r PyOptBookMeijiUnivBA/requirements.txt -q

In [ ]:
import itertools

import pandas as pd
import pulp

In [ ]:
# データ読み込み
df_student = pd.read_csv("student.csv", index_col=0)
df_pref = pd.read_csv("pref.csv", index_col=0)

# データ定義
S = df_student.index.to_list()
Z = df_pref.columns.to_list()
scores = {}
for s, score in zip(S, df_student.score.to_list()):
    scores[s] = score

# 学生とゼミのペアのリスト
SZ = [(s, z) for s in S for z in Z]

# 学生とゼミの志望度のペアがキー
d = df_pref.transpose().to_dict()
P = {(s, z): d[s][z] for s in S for z in Z}

# 各ゼミの志望順位ごとの学生数
df_counts = pd.DataFrame(index=[0, 1, 2, 3, 4,5])
for z in Z:
    df_counts[z] = df_pref[z].value_counts().sort_index()

# 後で使う関数の作成
def get_result_df(x_result):
    # 結果を格納したデータフレームを作成する
    df_result = df_pref.copy()
    df_result["assigned_zemi"] = None
    df_result["pref"] = 0
    df_result["score"] = df_student.score
    for s, z in itertools.product(S, Z):
        if x_result[s, z] == 1:
            df_result.loc[s, "assigned_zemi"] = z
            df_result.loc[s, "pref"] = int(df_pref.loc[s, z])
    return df_result

# データ確認

In [ ]:
df_student.head()

In [ ]:
df_student.hist()

In [ ]:
df_student.gender.value_counts()

In [ ]:
df_pref.head()

In [ ]:
# 各ゼミの志望順位ごとの学生数
df_counts

# 最適化例 （第1志望の割り当て数を増やす）

In [ ]:
prob = pulp.LpProblem('ClassAssignmentProblem', pulp.LpMaximize)
x = pulp.LpVariable.dicts("x", SZ, cat="Binary")

# 学生が割り当てるのは多くとも一つのゼミ
for s in S:
    prob += pulp.lpSum([x[s, z] for z in Z]) <= 1

# 学生の志望度が0のゼミには割り当てない
for (s,z), pref in P.items():
    if pref == 0:
        prob += x[s, z] == 0

# 各ゼミの最大人数は10人
for z in Z:
    prob += pulp.lpSum([x[s,z] for s in S]) <= 10

# ゼミに割り当てられた学生の数の総和を最大にする
prob += sum([sum([(6-P[s,z]) * x[s, z] for s in S]) for z in Z])
# 求解
status = prob.solve()
print('Status:', pulp.LpStatus[status])

x_result = {}
for s, z in itertools.product(S, Z):
    x_result[s,z] = int(x[s,z].value())

In [ ]:
df_result = get_result_df(x_result)
df_result.head()

In [ ]:
# 60%以上が第一志望に内定
df_result.pref.value_counts()

In [ ]:
# 第一志望がzemi07で成績が高いにも関わらず内定できていない人がたくさんいる
df_result[df_result.zemi07 ==1].sort_values(by="score", ascending=False)

# 成績の高い順に割り当てる

In [ ]:
# 成績の高い順の学生のリスト
sorted_students = df_student.sort_values(by="score", ascending=False).index.to_list()

In [ ]:
y = {}
zemi_count = { zemi: 0 for zemi in Z}
for s in sorted_students:
    for z in Z:
        y[(s,z)]= 0
    # 学生sの志望順のゼミのリスト
    zemis = df_pref.loc[s].sort_values(ascending=False)[:5][::-1].index.to_list()
    for zemi in zemis:
        if zemi_count[zemi] <= 9:
            y[s, zemi] = 1
            zemi_count[zemi] += 1
            break

In [ ]:
# 結果のデータフレーム取得
df_result = get_result_df(y)

In [ ]:
# 配属されない人が10人いる
df_result.pref.value_counts()

In [ ]:
# 公平性は増している
df_result[df_result.zemi07 ==1].sort_values(by="score", ascending=False)